In [1]:
%pip install arcgis

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import requests
import json
import webbrowser
from arcgis.gis import GIS
from getpass import getpass

In [7]:
pwd = getpass()
gis = GIS(username="thabo.traore_AAMU", password=pwd)
token = gis._con.token

In [ ]:
survey_api_url = "https://survey123.arcgis.com/api/surveys"
portalUrl = "https://aamu.maps.arcgis.com"

In [14]:
search = "search"

payload = {
    "searchString": "City of Rocky Mount Service Line Material Survey",
    "searchSurveyType": "owned",
    "isPublished": "True",
    "token": token,
    "portalUrl": portalUrl
}

request_url = '{0}{1}'.format(survey_api_url, search)
r = requests.get(request_url, params=payload)

if r.status_code == requests.codes.ok:
    print("...Results count: ", json.loads(r.content))
    for search_result in json.loads(r.content):
        print("\n...Search result: \n", search_result)
else:
    r.raise_for_status()

...Results count:  {'errorCode': 101, 'errorMessage': 'Unknown service'}

...Search result: 
 errorCode

...Search result: 
 errorMessage


In [15]:
%pip install requests pandas reportlab


   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------- ----------------------------- 0.5/1.9 MB 17.5 MB/s eta 0:00:01
   ---------------------------------------- 1.9/1.9 MB 4.9 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import requests
import pandas as pd
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

# Replace these with your ArcGIS details
FEATURE_LAYER_URL = "https://services5.arcgis.com/s1DFLqWfkSIcXZ9O/arcgis/rest/services/survey123_ff660c6fae384e3fa1082b0a708555ba_form/FeatureServer/0/query"
TOKEN = "mzFcMRqhxzPAoRJavp2MJg-w3Fau_D0Ix0Qviif4Gs2B6QMLbfdRhemy-rS5IGGgZCF2xD3K_WcpdZ8nn-17hTyiHE0Z7B_FswkDRreJWAjcXXNOhf-yDJwKqhVMSuzL4jBJcy1wTJ8sB72ab79kPF1K3wgnLpyLYblmWRVgee5huCcufZ0F_vMJ9tJ_vdVJ"

# Function to fetch data from the feature layer
def fetch_arcgis_data():
    params = {
        "where": "1=1",  # Get all records
        "outFields": "*",  # Fetch all fields
        "f": "json",  # Format response as JSON
        "token": TOKEN  # Authentication token
    }

    response = requests.get(FEATURE_LAYER_URL, params=params)
    
    if response.status_code == 200:
        data = response.json()
        features = data.get("features", [])
        return [feature["attributes"] for feature in features]
    else:
        print("Error fetching data:", response.text)
        return []

# Function to create a PDF report
def generate_pdf_report(data, filename="ArcGIS_Report.pdf"):
    c = canvas.Canvas(filename, pagesize=letter)
    c.setFont("Helvetica", 12)
    
    c.drawString(100, 750, "ArcGIS Feature Layer Report")
    c.drawString(100, 730, f"Total Records: {len(data)}")
    
    y_position = 700
    for index, row in enumerate(data[:20]):  # Show only first 20 rows
        c.drawString(100, y_position, f"Record {index + 1}: {row}")
        y_position -= 20
        if y_position < 50:  # Create a new page if needed
            c.showPage()
            y_position = 750

    c.save()
    print(f"PDF report saved as {filename}")

# Main execution
if __name__ == "__main__":
    records = fetch_arcgis_data()
    if records:
        df = pd.DataFrame(records)
        print(df.head())  # Display some data
        generate_pdf_report(records)
    else:
        print("No data retrieved.")


In [ ]:
import requests
import pandas as pd
from reportlab.lib.pagesizes import letter, landscape
from reportlab.lib import colors
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle

# Replace these with your ArcGIS details
FEATURE_LAYER_URL = "https://services5.arcgis.com/s1DFLqWfkSIcXZ9O/arcgis/rest/services/survey123_ff660c6fae384e3fa1082b0a708555ba_form/FeatureServer/0/query"
TOKEN = "mzFcMRqhxzPAoRJavp2MJg-w3Fau_D0Ix0Qviif4Gs2B6QMLbfdRhemy-rS5IGGgZCF2xD3K_WcpdZ8nn-17hTyiHE0Z7B_FswkDRreJWAjcXXNOhf-yDJwKqhVMSuzL4jBJcy1wTJ8sB72ab79kPF1K3wgnLpyLYblmWRVgee5huCcufZ0F_vMJ9tJ_vdVJ"

# Function to fetch data from the feature layer
def fetch_arcgis_data():
    params = {
        "where": "1=1",  # Get all records
        "outFields": "*",  # Fetch all fields
        "f": "json",  # Format response as JSON
        "token": TOKEN  # Authentication token
    }

    try:
        response = requests.get(FEATURE_LAYER_URL, params=params, timeout=15)
        response.raise_for_status()  # Raise HTTP error if one occurs
        data = response.json()

        # Check for ArcGIS API errors
        if "error" in data:
            print("ArcGIS API Error:", data["error"])
            return []

        features = data.get("features", [])
        return [feature["attributes"] for feature in features]

    except requests.exceptions.RequestException as e:
        print("Request error:", e)
        return []

# Function to create a well-formatted PDF report
def generate_pdf_report(data, filename="ArcGIS_Report.pdf"):
    if not data:
        print("No data available for the report.")
        return

    # Convert data to a Pandas DataFrame for better handling
    df = pd.DataFrame(data)

    # Define columns to include in the report
    columns_to_include = [
        "first_name", "last_name", "property_address", "phone_number",
        "email", "who_will_conduct_the_material_l", "certification",
        "type_of_occupancy", "type_of_property"
    ]

    # Rename columns to match your dataset headers
    column_headers = {
        "first_name": "First Name",
        "last_name": "Last Name",
        "property_address": "Property Address",
        "phone_number": "Phone Number",
        "email": "Email",
        "who_will_conduct_the_material_l": "Who Conducts Test?",
        "certification": "Certification",
        "type_of_occupancy": "Occupancy Type",
        "type_of_property": "Property Type"
    }

    # Keep only selected columns
    df = df[columns_to_include]
    df.rename(columns=column_headers, inplace=True)

    # Convert DataFrame to list format for the report
    table_data = [df.columns.tolist()] + df.values.tolist()

    # Create PDF document (Landscape for better table fit)
    pdf = SimpleDocTemplate(filename, pagesize=landscape(letter))
    table = Table(table_data, repeatRows=1)

    # Add styling
    style = TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
        ('GRID', (0, 0), (-1, -1), 1, colors.black),
        ('WORDWRAP', (0, 0), (-1, -1)),  # Wrap text if too long
    ])
    table.setStyle(style)

    # Build and save the PDF
    pdf.build([table])
    print(f"✅ PDF report saved as {filename}")

# Main execution
if __name__ == "__main__":
    records = fetch_arcgis_data()
    if records:
        df = pd.DataFrame(records)
        print(df.head())  # Display sample data in console
        generate_pdf_report(records)
    else:
        print("No data retrieved.")


   objectid                              globalid   CreationDate  \
0         1  baca359a-83de-472a-8246-b856b942dc51  1741810191347   
1         2  93ed02e6-a54b-4326-86c4-e7c0215b58a5  1741810534716   
2         3  bdeb1baa-3994-4119-8df9-bf33172dfa66  1741810702963   
3         4  ffbde5d2-ea5c-4dbe-846c-58e3b5610876  1741810829016   
4         5  f7c9300d-6982-4687-8f3b-5662cc524c15  1741810928831   

             Creator       EditDate             Editor  \
0  thabo.traore_AAMU  1741810191347  thabo.traore_AAMU   
1  thabo.traore_AAMU  1741810534716  thabo.traore_AAMU   
2  thabo.traore_AAMU  1741810702963  thabo.traore_AAMU   
3  thabo.traore_AAMU  1741810829016  thabo.traore_AAMU   
4  thabo.traore_AAMU  1741810928831  thabo.traore_AAMU   

   who_will_conduct_the_material_l certification first_name  last_name  \
0  Myself, the property owner or m       I agree    Ibrahim     Traore   
1  Myself, the property owner or m       I agree       Juan      Perez   
2  Myself, the prope